# **Reinforcement Learning Sprint Challenge - play Taxi**
For the sprint challenge, we will apply the techniques we have learned to play Taxi, an environment in the OpenAI Gym. In this task the agent controls a taxi that can navigate between four locations. The goal is to pick up a passenger from one location and drop them off to another. You get 20 points for each successful drop off, but lose 1 point for each step you take, and additionally there is a 10 point penalty for illegal pick-up/drop-off actions.

You can create the environment and watch a random agent play with this code:

In [0]:
!pip install gym

In [0]:
import numpy as np
import random
import time
import gym
from gym import wrappers

In [0]:
env = gym.make('Taxi-v2')
state = env.reset()
env.render()
print(env.step(action))

total_reward = 0
done = False
while not done:
    state, reward, done, info = env.step(env.action_space.sample())
    total_reward += reward
    env.render()
    print(env.step(action))
    print('Cumulative reward:', total_reward, '\n')

print('Total reward:', total_reward)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

(27, -10, False, {'prob': 1.0})
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
(27, -10, False, {'prob': 1.0})
Cumulative reward: -10 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
(7, -10, False, {'prob': 1.0})
Cumulative reward: -11 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
(7, -10, False, {'prob': 1.0})
Cumulative reward: -21 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
(27, -10, False, {'prob': 1.0})
Cumulative reward: -22 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
(7, -10, False, {'prob': 1.0})
Cumulative reward: -23 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
(7, -10, False, {'prob': 1.0})
Cumu

You'll see that a random agent doesn't do very well - in a trial run the score reached -713 before the environment terminated.

# **Instructions**
Make a Python notebook where you work on the below goals. You can use whatever environment you wish to develop, but for turning in you should add the file to the ML-Reinforcement-Learning repository in the sprintchallenge/ directory. Add, commit, push, and it will appear in your already open pull request.

The goals involve trying to beat a score in Taxi - be sure to measure the score of your approach after it is trained, and not during the training. This snippet measures performance (run a simulation repeatedly and average total rewards):

In [0]:
for episode in range(episodes):
    state = env.reset()  # Assuming you already have env created as above
    total_rewards = 0
    
    for step in range(max_steps):
        action = env.action_space.sample()  # TODO your policy here!
        state, reward, done, info = env.step(env.action_space.sample())
        total_rewards += reward
        if done:
            break
    rewards.append(total_rewards)        

print('Average score over time:', sum(rewards) / episodes)

Average score over time: -387.148


# **Goal 1 - Beat Random**
As an initial goal, come up with an agent/policy that does better than random. And more specifically, try to at least have a positive score (>0) average.

This game is discrete, and so you can use the Q-learning approach and build a matrix of states by actions populated with expected rewards. This approach should work well and it is suggested you start with it.

In [0]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


There are 6 possible actions and 500 possible states. The actions are to go south, west, north, east, pick up the passenger, and drop off the passenger. The states are the 25 locations on the grid times the possible passenger locations (5: each of the destinations plus in the taxi) times the number of destinations (4).

In [0]:
# Initialize Q-table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [0]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.125       # learning rate (0 < alpha <= 1) - how much Q-values update per iteration
gamma = 0.65      # discount factor (0 <= gamma <= 1) - how much future rewards are discounted
epsilon = 0.01      #

# Plotting metrics
all_epochs = []
all_penalties = []
total_reward = 0
rewards = []

for i in range(1, 10001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        total_reward += reward
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episodes: {i}")

print("Training finished.\n")
print("Total rewards:", total_reward)
print("Average rewards:", total_reward / i)

Episodes: 10000
Training finished.

Total rewards: 80073
Average rewards: 8.0073
CPU times: user 3.47 s, sys: 302 ms, total: 3.77 s
Wall time: 3.64 s


In [0]:
q_table[0:5]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.63022799e+00, -9.49568408e-01, -1.62440781e+00,
        -9.74550511e-01,  8.41938772e-02, -9.96389618e+00],
       [ 2.05885936e-02,  1.60520479e+00,  2.03995004e-03,
         1.62358665e+00,  4.10460089e+00, -7.43091036e+00],
       [-2.05068772e+00, -1.61947154e+00, -2.05493319e+00,
        -1.61744124e+00, -9.45273980e-01, -1.06321542e+01],
       [-2.63528417e+00, -2.65279519e+00, -2.63528489e+00,
        -2.63537936e+00, -1.10424937e+01, -1.06412247e+01]])

# **Goal 2 - Beat Basic Q-learning**
Once you've got an initial Q-learning approach working, you should try to improve it via hyperparameter optimization. A score (average performance across many games) generated without optimizing hyperparameters that you should try to beat: 8.467

You should be able to do better without having to use different techniques (i.e. just with hyperparameter optimization).

In [0]:
# The following is from Moustafa Alzantot's article on Genetic algorithms for 
# hyperparameter tuning

# I was unable to tweak it sufficiently in time to apply to this sprint challenge

# -----------------------------------------------
# def run_episode(env, policy, episode_len=100):
#     total_reward = 0
#     obs = env.reset()
#     for t in range(episode_len):
#         # env.render()
#         action = policy[obs]
#         obs, reward, done, _ = env.step(action)
#         total_reward += reward
#         if done:
#             # print('Epside finished after {} timesteps.'.format(t+1))
#             break
#     return total_reward


# def evaluate_policy(env, policy, n_episodes=100):
#     total_rewards = 0.0
#     for _ in range(n_episodes):
#         total_rewards += run_episode(env, policy)
#     return total_rewards / n_episodes

# def gen_random_policy():
#     return np.random.choice(6, size=((500)))

# def crossover(policy1, policy2):
#     new_policy = policy1.copy()
#     for i in range(16):
#         rand = np.random.uniform()
#         if rand > 0.5:
#             new_policy[i] = policy2[i]
#     return new_policy

# def mutation(policy, p=0.05):
#     new_policy = policy.copy()
#     for i in range(16):
#         rand = np.random.uniform()
#         if rand < p:
#             new_policy[i] = np.random.choice(4)
#     return new_policy

# if __name__ == '__main__':
#     random.seed(1234)
#     np.random.seed(1234)
#     env = gym.make('Taxi-v2')
#     env.seed(0)
#     ## Policy search
#     n_policy = 100
#     n_steps = 20
#     start = time.time()
#     policy_pop = [gen_random_policy() for _ in range(n_policy)]
#     for idx in range(n_steps):
#         policy_scores = [evaluate_policy(env, p) for p in policy_pop]
#         print('Generation %d : max score = %0.2f' %(idx+1, max(policy_scores)))
#         policy_ranks = list(reversed(np.argsort(policy_scores)))
#         elite_set = [policy_pop[x] for x in policy_ranks[:5]]
#         select_probs = np.array(policy_scores) / np.sum(policy_scores)
#         child_set = [crossover(
#             policy_pop[np.random.choice(range(n_policy), p=select_probs)], 
#             policy_pop[np.random.choice(range(n_policy), p=select_probs)])
#             for _ in range(n_policy - 5)]
#         mutated_list = [mutation(p) for p in child_set]
#         policy_pop = elite_set
#         policy_pop += mutated_list
#     policy_score = [evaluate_policy(env, p) for p in policy_pop]
#     best_policy = policy_pop[np.argmax(policy_score)]

#     end = time.time()
#     print('Best policy score = %0.2f. Time taken = %4.4f'
#             %(np.max(policy_score), (end-start)))    

#     ## Evaluation
#     env = wrappers.Monitor(env, '/tmp/frozenlake1', force=True)
#     for _ in range(200):
#         run_episode(env, best_policy)
#     env.close()
# gym.upload('/tmp/frozenlake1', api_key=...)

# **Goal 3 - Beat Optimized Q-learning (stretch)**
Now the sky's the limit - or rather, the best possible performance in Taxi. With the default environment, optimized Q-learning has achieved an average score of 9.423. See if you can get in that range, or possibly even beat it, by employing alternative techniques.

What is an alternative technique? It's anything that maps from environment state to action - Q-learning achieves this by populating a Q-table, but any model that can take the environment state and possible action as input and give predicted reward as output can serve the same purpose. And the Gym environment object gives us a simulator perfect for generating arbitrary amounts of training data to train such a model.

The true optimal performance in Taxi is probably not much more than the optimized Q-learning score - the score certainly has to be less than 20 as the taxi must always take at least some steps to achieve the task.

If you get this far, feel free to share the best score you get, and see how your classmates are doing. Good luck!